In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from scipy import interpolate
from scipy import signal
import datetime

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [ ]:
# Code from https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units
def butter_lowpass(cutoff, sample_freq, order=5):
    nyq = 0.5 * sample_freq 
    normal_cutoff = cutoff / nyq #converts from frequency to cycles per half sample
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def get_sample_freq(time_stamps):
    mili = time_stamps[1] - time_stamps[0]
    seconds = mili/1e3
    return 1/seconds

def corr_dist(u, v):
    centered_u = u - np.mean(u)
    centered_v = v - np.mean(v)
    return 1 - centered_u.T@centered_v/(np.linalg.norm(centered_u)*np.linalg.norm(centered_v))

In [ ]:
accel_raw = pd.read_csv("WISDM_at_v2.0/WISDM_at_v2.0_raw.txt", sep = ";|,", engine='python')  
accel_raw.columns = ["id", "tag", "time", "a_x", "a_y", "a_z", "to_drop"]
accel_raw = accel_raw.drop("to_drop",1)
id_list = accel_raw.id.unique()


In [ ]:

labels = pd.read_csv("WISDM_at_v2.0/WISDM_at_v2.0_demographics.txt", sep = ";|,", engine='python')  
labels.columns = ["id", "height", "sex", "age", "weight", "leg_injury"]
labels_id_list = labels.id.unique()
#id_list = labels.id.unique()

In [ ]:

counter = 0

for id_val in id_list:
    if id_val in labels_id_list:
        counter += 1
print(counter)
    

In [ ]:
extracted_gaits = []
gait_labels = []
#fs = get_sample_freq(accel["time"].values)  # sample rate, Hz
fs = 20
for k in range(2):#len(id_list)): 
    accel = accel_raw.loc[accel_raw['id'] == id_list[k]]

    # Make vector with magnitude of accelerations for each data point
    accel_mag = np.linalg.norm(accel[["a_x","a_y", "a_z"]].values, axis = 1)

    # Filter requirements.
    order = 6
    
    cutoff = 3  # desired cutoff frequency of the filter, Hz

    # Time stamps in seconds
    t = accel["time"].values/1e3 - accel["time"].values[0]/1e3

    # "Noisy" data.
    data = accel_mag

    # Filter the data, and plot both the original and filtered signals.
    filtered_data = butter_lowpass_filter(data, cutoff, fs, order)
    # Global max of filtered data 
    #print(len(filtered_data))
    if len(filtered_data) > 200:
        # Note: different from paper because min gives zero
        i_tilde = np.argmax(filtered_data[50:-80]) + 50

        # Max within 10 observations of i_tilde of unfiltered data
        # Why do we do - 10 here???
        i_star = np.argmax(accel_mag[i_tilde-10:i_tilde+10]) - 10 + i_tilde

        # Finding template
        # Acceleration within a half second on either side of i_star
        template_size =2*np.floor(fs*0.5).astype(int)
        #print("This is template size: ", template_size)
        template = accel_mag[i_star - template_size//2 : i_star + template_size//2]
        xyz_accel = accel[["a_x", "a_y", "a_z"]].values[i_star - template_size//2 : i_star + template_size//2,:]
        template_updates = 30

        psi_threshold = .4
        window_size = 3*template_size
        i_min = 0
        i_max = window_size
        alpha = 0.85 #this probably needs to change
        psi_min_vals = []

        original_template = template

        j = 0


        # Plot original template
        t = np.linspace(0, template_size, template_size)
        #     plt.plot(t, template, 'g-', linewidth=2, label='Template')
        #     plt.title("Original Template")
        #     plt.show()
        while j < template_updates:
            # Make sure we don't index out of range
            if i_max >= accel_mag.shape[0] - template_size:
                break

            # Update psi values for current template
            psi_vals = np.zeros(window_size)
            for i in range(i_min, i_max):
                psi_vals[i - i_min] = corr_dist(template, accel_mag[i:i+template_size])

              # Plot psi_vals
            """
            plt.figure()
            psi_t = np.linspace(i_min, i_max, i_max-i_min)
            plt.figure()
            plt.plot(psi_t, psi_vals, 'p-')
            plt.title('Psi Values')
            plt.xlabel('Time')
            plt.ylabel('Psi Value')
            """
            # Find first psi value below psi_threshold between i_min and i_max
            psi_min = np.argmin(psi_vals)


            t_prime = accel_mag[i_min+psi_min:i_min + psi_min + template_size]
            if psi_vals[psi_min] < psi_threshold:
                template = template*alpha + (1 - alpha)*t_prime
                j += 1
                psi_min_vals.append(psi_vals[psi_min])
                # Plot progress of template
        #         plt.plot(t, template, 'b-', linewidth=2, label='Template')
        #         plt.plot(t, t_prime, 'r-', linewidth=2, label='Data')
        #         plt.legend()
        #         plt.title("Template Progress")
        #         plt.show()

            i_min += window_size
            i_max += window_size

    #     plt.close()
    #     plt.plot(t, template, 'b-', label = "Final Template")
    #     plt.legend()
    #     plt.title("Final Template")
    #     plt.show()
        template_func = interpolate.interp1d(range(len(template)), template)
        x_new = np.linspace(0, len(template)-1, 5*(len(template)))
        print(x_new)
        print(len(template))
        upsampled_template = template_func(x_new)
        plt.plot(np.linspace(0, len(template)-1, 5*len(template)), upsampled_template, 'r.')
        plt.plot(range(len(template)), template, 'g.')
        plt.show()
        #label = labels.loc[labels['id'] == id_list[k]]
        #print(label)
        extracted_gaits.append(np.concatenate((np.resize(template, (20, 1)), xyz_accel), axis=1))
        
def upsample(data, factor):
    to_up_func = interpolate.interp1d(range(len(data)), data)
    
